In [1]:
import numpy as np
import pandas as pd
import requests
import json
query = 'query=title-abs-key' # There are other parameters available that can be added to query based on Scopus instruction
date  = 'date=1990-2020'
query = query+date
# specifying API, it can be obtined from Scopus website 
apikey='your api key'
api_resource = "https://api.elsevier.com/content/search/scopus"


In [2]:
# helper function for saerching engine
def _Search_Engine_Scopus(apikey, query, type_, view, index=0):

    par = {'apikey': apikey, 'query': query, 'start': index,
           'httpAccept': 'application/json', 'view': view}
    if type_ == 'article' or type_ == 1: # both of these values are for the articles based on Scopus website information
        r = requests.get(api_resource, params=par)
    js = r.json()
    total_count = int(js['search-results']['opensearch:totalResults'])
    entries = js['search-results']['entry']
# creating pandas dataframe to structure our entries
    result_df = pd.DataFrame(entries) 
    if index == 0:
        return(result_df, total_count)
    else:
        return(result_df)

In [3]:
class Search_Engine(object):
    def __init__(self, apikey='your api key'):
        self.apikey = apikey
    def add_key(self, apikey):
        self.apikey = apikey
#creating function and loops for retriving data, there are different amount of counts, specifying on Scopus website, 25 is for basic account but 100 that I used is for permimum account 
    def search(self, query, count=100, type_=1, view='COMPLETE'):
        result_df, total_count = _Search_Engine_Scopus(self.apikey, query, type_, view=view)
#here, we have to create an if loop inside a while loop to retrieve all instances we want without facing limitations.         
        if total_count <= count:
            count = total_count
        i = 1
        while True:
            index = 25*i
            result_df = result_df.append(_Search_Engine_Scopus(self.apikey, query, type_, view=view, index=index),
                                         ignore_index=True)
            if result_df.shape[0] >= count:
                return result_df[:count]
            i += 1

In [4]:
Scopus_wrapper=Search_Engine(apikey)
final_df=Scopus_wrapper.search("(big data)", count=3001)

In [ ]:
##There is another way in order to do what I have done above without creating class.
##creating function and loops for retriving data, there are different amount of counts, specifying on Scopus website, 25 is for basic account but 100 that I used is for permimum account 
#def search(apikey, query, count=100, type_=1, view='COMPLETE'):
#    result_df, total_count = _Search_Engine_Scopus(apikey, query, type_, view=view)
##here, we have to create an if loop inside while loop to retrive all instances that we want
#    if total_count <= count:
#        count = total_count
#    i = 1
#    while True:
#        index = 25*i
#        result_df = result_df.append(_Search_Engine_Scopus(apikey, query, type_, view=view, index=index),
#                                         ignore_index=True)
#        if result_df.shape[0] >= count:
#            return result_df[:count]
#        i += 1
#final_df = search(apikey,"(big data)", count=3001)

In [5]:
#Claening section
final_df = final_df.drop(columns=['@_fa','link','eid','prism:issn','prism:eIssn','prism:volume',
                          'prism:pageRange','prism:coverDate','prism:coverDisplayDate',
                          'pubmed-id','author-count','article-number','source-id','fund-acr',
                          'fund-no','fund-sponsor','freetoread','prism:issueIdentifier','pii',
                                  'prism:isbn','author','affiliation','freetoreadLabel','dc:identifier'])
final_df=final_df.rename(columns={'prism:url': 'url', 'dc:title': 'title','dc:creator':'creator',
                                 'prism:publicationName':'publicationName','prism:doi':'DOI',
                                'dc:description':'description','prism:aggregationType':'aggregationType'
                                 })



In [6]:
final_df

,url,title,creator,publicationName,DOI,description,citedby-count,aggregationType,subtype,subtypeDescription,authkeywords,openaccess,openaccessFlag
0,https://api.elsevier.com/content/abstract/scop...,Ameliorative potential role of Rosmarinus offi...,Morsi R.M.,Brazilian Journal of Biology,10.1590/1519-6984.258234,The present work was showed to assess the effe...,0,Journal,ar,Article,chemotherapy | etoposide | liver | rat | rosemary,1,True
1,https://api.elsevier.com/content/abstract/scop...,Drought variability assessment using standardi...,Qaisrani Z.N.,Brazilian Journal of Biology,10.1590/1519-6984.261001,Drought variability analysis is of utmost conc...,0,Journal,ar,Article,drought variability | precipitation deciles | ...,1,True
2,https://api.elsevier.com/content/abstract/scop...,"Assessment of land use cover changes, carbon s...",Raqeeb A.,Brazilian Journal of Biology,10.1590/1519-6984.253821,Land use and land cover change are affecting t...,1,Journal,ar,Article,Carbon sequestration | Carbon stock | Land use...,1,True
3,https://api.elsevier.com/content/abstract/scop...,Reduced genetic diversity and the success of t...,Luz S.C.S.,Brazilian Journal of Biology,10.1590/1519-6984.248656,Several species of Cichla successfully coloniz...,0,Journal,ar,Article,Bottleneck | Cichlid | Low genetic diversity |...,1,True
4,https://api.elsevier.com/content/abstract/scop...,A Label-Free Optical Nanobiosensor for Measure...,Semyari M.,Biointerface Research in Applied Chemistry,10.33263/BRIAC133.291,Access to accurate diagnostic tools is one of ...,0,Journal,ar,Article,aptamer | biosensor | colorimetric assay | glu...,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,https://api.elsevier.com/content/abstract/scop...,A machine learning approach to map crystal ori...,Wittwer M.,npj Computational Materials,10.1038/s41524-021-00688-1,Mapping grain orientation in crystalline solid...,4,Journal,ar,Article,NaN,1,True
2997,https://api.elsevier.com/content/abstract/scop...,A universal methodology for reliable predictin...,Rezaei T.,Scientific Reports,10.1038/s41598-022-04942-4,Understanding the drug solubility behavior is ...,2,Journal,ar,Article,NaN,1,True
2998,https://api.elsevier.com/content/abstract/scop...,A crowdsourced global data set for validating ...,See L.,Scientific Data,10.1038/s41597-021-01105-4,Several global high-resolution built-up surfac...,1,Journal,dp,Data Paper,NaN,1,True
2999,https://api.elsevier.com/content/abstract/scop...,Model-based evaluation of alternative reactive...,Liu Q.H.,Nature Communications,10.1038/s41467-021-27939-5,There are contrasting results concerning the e...,3,Journal,ar,Article,NaN,1,True
